In [1]:
import numpy as np
import pandas as pd
import os

from ipynb.fs.full.tuneSvmForDeconv import tuneSvmForDeconv

In [2]:
# Function nuSvrR
# ----------------
# @method nuSvrR
# @param [DataFrame] Signature
# @param [DataFrame] Expression
# @param [array] nuseq
# @param [float] delta
# @return -------------
def nuSvrR(X, Y, nuseq = [0.25,0.5,0.75], delta = 0.007):

    wsel = [1 for x in range(len(X.columns))]
    wsel = pd.DataFrame(wsel, X.columns).T
    ok = True
    i = 0
    
    while ok:
        i = i + 1
        # Run function
        XX = X.loc[:, X.columns.isin(wsel.columns[wsel.values[0] > 0])]
        model = tuneSvmForDeconv(X = XX, Y = Y, nuseq = [0.25,0.5,0.75], delta = 0.007)
        # Get betas
        w = model.coef_
        # Set values i to zero where i < 0
        w = np.where(w<0, 0, w)
        absW = w
        
        # Normalize data
        w = w/np.sum(w)
        
        # Set values i to zero where i < delta
        w = np.where(w<delta, 0, w)
        
        # Checking if all the values are NaN
        if w.isnull().all():
            return False
        
        # Checking if any value is < delta
        if w.apply(lambda x: x < delta).any().any():
            if(w.apply(lambda x: x > 0).sum().sum() == 1):
                break
            wsel.loc[-wsel.columns.isin(w.columns[w.values >= delta])] = 0
    